In [1]:
print("ok")

ok


In [2]:
%pwd

'c:\\Users\\Saif Khan\\AI-Projects\\End-to-End-Medical-Chatbot-Generative-AI\\research'

In [3]:
import os
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\Saif Khan\\AI-Projects\\End-to-End-Medical-Chatbot-Generative-AI'

In [8]:
from langchain.document_loaders import DirectoryLoader, PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [9]:
#Extract the data from the PDF file
def load_pdf_file(data):
    loader= DirectoryLoader(data,
                            glob="*.pdf",
                            loader_cls=PyPDFLoader)
    
    documents=loader.load()

    return documents

In [10]:
extracted_data=load_pdf_file(data='Data/')

In [ ]:
#extracted_data

In [12]:
#Split the Data into Text Chunks
def text_split(extracted_data):
    text_splitter=RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)
    text_chunks=text_splitter.split_documents(extracted_data)
    return text_chunks

In [13]:
text_chunks=text_split(extracted_data)
print("Length of Text Chunks", len(text_chunks))

Length of Text Chunks 6970


In [ ]:
#text_chunks

In [110]:
from langchain_community.embeddings import HuggingFaceEmbeddings
from sentence_transformers import SentenceTransformer

In [111]:
#Download the Embeddings from Hugging Face
def download_hugging_face_embeddings():
    embeddings=HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
    return embeddings

In [103]:
embeddings = download_hugging_face_embeddings()

In [104]:
query_result = embeddings.embed_query("Hello World")
print("Length", len(query_result))

Length 384


In [ ]:
#query_result

In [60]:
from dotenv import load_dotenv
load_dotenv()

True

In [107]:
PINECONE_API_KEY=os.environ.get('PINECONE_API_KEY')
GOOGLE_API_KEY=os.environ.get('GOOGLE_API_KEY')

In [62]:
from pinecone.grpc import PineconeGRPC as Pinecone
from pinecone import ServerlessSpec
import os

pc = Pinecone(api_key=PINECONE_API_KEY)

index_name = "medicalbot"


pc.create_index(
    name=index_name,
    dimension=384,
    metric="cosine",
    spec=ServerlessSpec(
        cloud="aws",
        region="us-east-1"
    )
)

{
    "name": "medicalbot",
    "metric": "cosine",
    "host": "medicalbot-smsc2is.svc.aped-4627-b74a.pinecone.io",
    "spec": {
        "serverless": {
            "cloud": "aws",
            "region": "us-east-1"
        }
    },
    "status": {
        "ready": true,
        "state": "Ready"
    },
    "vector_type": "dense",
    "dimension": 384,
    "deletion_protection": "disabled",
    "tags": null
}

In [63]:
import os
os.environ["PINECONE_API_KEY"] = PINECONE_API_KEY
os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY

In [64]:
from langchain_pinecone import PineconeVectorStore
# Embed each chunk and upsert the embeddings into your Pinecone index
docsearch = PineconeVectorStore.from_documents(
    documents=text_chunks,
    index_name=index_name,
    embedding=embeddings,
)

In [65]:
# Load Existing index

from langchain_pinecone import PineconeVectorStore
# Embed each chunk and upsert the embeddings into your Pinecone index
docsearch = PineconeVectorStore.from_existing_index(
    index_name=index_name,
    embedding=embeddings
) 

In [67]:
docsearch

In [68]:
retriever = docsearch.as_retriever(search_type="similarity", search_kwargs={"k":3})

In [69]:
retrieved_docs = retriever.invoke("What is Acne?")

In [70]:
retrieved_docs

[Document(id='805b9c19-626f-4970-a3e2-e2ee37e86414', metadata={'author': '', 'creationdate': '2017-05-01T10:37:35-07:00', 'creator': '', 'keywords': '', 'moddate': '2017-05-01T10:37:35-07:00', 'page': 425.0, 'page_label': '426', 'producer': 'GPL Ghostscript 9.10', 'source': 'Data\\medical_book.pdf', 'subject': '', 'title': '', 'total_pages': 759.0}, page_content='Corticosteriod—A group of synthetic hormones\nthat are used to prevent or reduce inflammation.\nToxic effects may result from rapid withdrawal after\nprolonged use or from continued use of large doses.\nPatch test—A skin test that is done to identify aller-\ngens. A suspected substance is applied to the skin.\nAfter 24–48 hours, if the area is red and swollen,\nthe test is positive for that substance. If no reaction\noccurs, another substance is applied. This is con-'),
 Document(id='1a67e3ae-30fc-4352-aabc-ce4261035272', metadata={'author': '', 'creationdate': '2017-05-01T10:37:35-07:00', 'creator': '', 'keywords': '', 'modda

In [83]:
from langchain_google_genai import ChatGoogleGenerativeAI
llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-flash",
    temperature=0.4,
    max_tokens=500
)

In [96]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate


system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know thge answer, say you that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
)


prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

In [97]:
question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

In [93]:
response = rag_chain.invoke({"input": "How to cure Cough?"})
print(response["answer"])

Most coughs resolve within 7-21 days, as they're often related to the common cold or flu.  For lingering coughs or those with blood, consult a doctor.  Home remedies like increasing fluids, humidified air, and honey tea may provide temporary relief.


In [106]:
response = rag_chain.invoke({"input": "What is Fever?"})
print(response["answer"])

A fever is a body temperature elevation above 100°F (37.8°C).  A healthy person's temperature fluctuates between 97°F (36.1°C) and 100°F (37.8°C), with an average of 98.6°F (37°C).  It's a natural response to fight off foreign substances.
